In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_scaled = StandardScaler().fit_transform(df_market_data[["price_change_percentage_24h",	
                                                               "price_change_percentage_7d",
                                                               "price_change_percentage_14d",
                                                               "price_change_percentage_30d",
                                                               "price_change_percentage_60d",
                                                               "price_change_percentage_200d",
                                                               "price_change_percentage_1y"]])                              

In [ ]:
# Create a DataFrame with the scaled data
market_scaled_df = pd.DataFrame(market_scaled, columns=["price_change_percentage_24h",	
                                                               "price_change_percentage_7d",
                                                               "price_change_percentage_14d",
                                                               "price_change_percentage_30d",
                                                               "price_change_percentage_60d",
                                                               "price_change_percentage_200d",
                                                               "price_change_percentage_1y"])


# Copy the crypto names from the original data
market_scaled_df["coin_id"] = df_market_data.index

# Set the coinid column as index
market_scaled_df = market_scaled_df.set_index("coin_id")

# Display sample data
market_scaled_df.head()

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for i in k:
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
    kmodel = KMeans(n_clusters=i)
# 2. Fit the model to the data using `df_market_data_scaled`
    kmodel.fit(market_scaled_df)
# 3. Append the model.inertia_ to the inertia list
    inertia.append(kmodel.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow = {"k":k, "inertia":inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow)
elbow_df

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_original = elbow_df.hvplot.line(
        x="k",
        y="inertia",
        title="Elbow - Original Data")
elbow_original

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 
Looking at the Elbow line graph, the best 𝑘value is 4. There is not much of a difference in the 
𝑘value beyond 4. This is also supported by the inertia values in the data frame. The difference in inertia between 
𝑘values 3 and 4 is 44, while the difference between kvalues 4 and 5 is only 10.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(market_scaled_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
clusters = model.predict(market_scaled_df)

# Print the resulting array of cluster values.
print(clusters)

In [ ]:
# Create a copy of the DataFrame
market_clusters = market_scaled_df.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
market_clusters["clusters"] = clusters

# Display sample data
market_clusters.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

scatter_org = market_clusters.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="clusters",
    hover_cols="coin_id",
    title="Clusters - Original Data")
    
scatter_org

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
pca_data  = pca.fit_transform(market_scaled_df)
# three principal components.

# View the first five rows of the DataFrame. 
pca_data[:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 0.89

In [ ]:
# Create a new DataFrame with the PCA data.

# Creating a DataFrame with the PCA data
df_market_data_pca = pd.DataFrame(pca_data, columns=["PC1", "PC2", "PC3"])

# Copy the crypto names from the original data
crypto = market_clusters.index

# Set the coinid column as index
df_market_data_pca = df_market_data_pca.set_index(crypto)

# Display sample data
df_market_data_pca.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for i in k:
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
    model = KMeans(n_clusters=i)
    model.fit(df_market_data_pca)
    inertia.append(model.inertia_)


In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_dict= {"k":k, "inertia":inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_dict)
elbow_df.head()

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_pca = elbow_df.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow PCA")

elbow_pca

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** best valuefor K is 4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No it does not differ. the best k value found using the original data is also 4.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
Kmodel = KMeans(n_clusters=4)

In [ ]:
# Fit the K-Means model using the PCA data
Kmodel.fit(df_market_data_pca)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
clusters = Kmodel.predict(df_market_data_pca)

# Print the resulting array of cluster values.
clusters

In [ ]:
# Create a copy of the DataFrame with the PCA data
pca_copy = df_market_data_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_copy["clusters"] = clusters

# Display sample data
pca_copy.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter_pca = pca_copy.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="clusters",
    title = "Scatter - PCA",
    hover_cols="coin_id")
scatter_pca

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
composite_original =(elbow_original + elbow_pca)
composite_original

In [ ]:
# Composite plot to contrast the clusters
composite_scatter = (scatter_org + scatter_pca)
composite_scatter

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:**
  * fewer features produce clusters that are tighter together, and outliers are easier to spot because they are farther from the clusters.
    